In [ ]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}


In [ ]:
ten = []
url = []
for k in range(1,50):
  r = requests.get(f'https://www.sciencedirect.com/browse/journals-and-books?page={k}&contentType=JL', headers = headers)
  soup = BeautifulSoup(r.text)
  datas = soup.find_all('a',class_='anchor js-publication-title anchor-default')
  for data in datas:
    url.append(data['href'])
    ten_list = data.find_all('span',class_='anchor-text')
    for i in ten_list:
      ten.append(i.text.strip())


In [ ]:
url_fix = []
for i in url:
  url_1 = 'https://www.sciencedirect.com' + i
  url_fix.append(url_1)
url_fix

['https://www.sciencedirect.com/journal/aace-clinical-case-reports',
 'https://www.sciencedirect.com/journal/aasri-procedia',
 'https://www.sciencedirect.com/journal/academic-pathology',
 'https://www.sciencedirect.com/journal/academic-pediatrics',
 'https://www.sciencedirect.com/journal/academic-radiology',
 'https://www.sciencedirect.com/journal/acc-current-journal-review',
 'https://www.sciencedirect.com/journal/accident-analysis-and-prevention',
 'https://www.sciencedirect.com/journal/accident-and-emergency-nursing',
 'https://www.sciencedirect.com/journal/accounting-forum',
 'https://www.sciencedirect.com/journal/accounting-management-and-information-technologies',
 'https://www.sciencedirect.com/journal/accounting-organizations-and-society',
 'https://www.sciencedirect.com/journal/accounting-theory-and-practice',
 'https://www.sciencedirect.com/journal/achievements-in-the-life-sciences',
 'https://www.sciencedirect.com/journal/acog-clinical-review',
 'https://www.sciencedirect.co

In [ ]:
len(ten)

4831

In [ ]:
online_issn=[]
issn = []
print_issn = []
linking_issn = []
impact = []
for i in url_fix:
  r1 = requests.get(i,headers = headers)
  soup1 = BeautifulSoup(r1.content,'html.parser')

  #issn
  try:
    issn_ = soup1.find('div', class_ = 'js-issn text-s').findAll('div')
    for itr in issn_:
      temp_value = itr.text
      text = temp_value.split(':')[0]
      code = temp_value.split(':')[1]
      if text == 'ISSN':
        issn.append(code)
      elif text == 'Print ISSN':
        print_issn.append(code)
      elif text == 'Online ISSN':
        online_issn.append(code)
      elif text == 'Linking ISSN':
        linking_issn.append(code)
  # case None
    max_len = max([len(issn),len(online_issn), len(print_issn), len(linking_issn)])
    if len(issn) < max_len:
      issn.append(None)
    if len(online_issn) < max_len:
      online_issn.append(None)
    if len(print_issn) < max_len:
      print_issn.append(None)
    if len(linking_issn) < max_len:
      linking_issn.append(None)
  except:
    issn.append(None)
    online_issn.append(None)
    print_issn.append(None)
    linking_issn.append(None)

  #impact
  impact_factor = soup1.find('div', class_ = 'js-impact-factor')
  impact_factor = impact_factor.find('span', class_='text-l u-display-block') if impact_factor else ""
  impact.append(impact_factor.getText() if impact_factor else "")

In [ ]:
import pandas as pd
data_document = pd.DataFrame.from_dict({
    'Ten tap chi': ten,
    'issn': issn,
    'online issn': online_issn,
    'print issn': print_issn,
    'linking issn':linking_issn,
    'impact_factor': impact
},orient='index')

In [ ]:
data_document = data_document.transpose()
data_document


,Ten tap chi,issn,online issn,print issn,linking issn,impact_factor
0,AACE Clinical Case Reports,2376-0605,None,None,None,
1,AASRI Procedia,2212-6716,None,None,None,
2,Academic Pathology,2374-2895,None,None,None,1
3,Academic Pediatrics,None,1876-2867,None,1876-2859,3.1
4,Academic Radiology,None,1878-4046,1076-6332,None,4.8
...,...,...,...,...,...,...
4826,"Zeszyty Naukowe WCO, Letters in Oncology Science",1734-0489,None,None,None,
4827,Zoological Journal of the Linnean Society,0024-4082,None,None,None,
4828,Der Zoologische Garten,0044-5169,None,None,None,
4829,Zoologischer Anzeiger,None,1873-2674,0044-5231,None,1.4


In [ ]:
data_document.to_csv('CrawlingData.csv')